In [1]:
import pandas as pd
from shutil import copyfile
from shutil import rmtree
from Bio import SeqIO
import os
import subprocess
from joblib import Parallel, delayed
import pickle
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import random
from functools import reduce
import operator
%matplotlib inline

In [2]:
from bokeh.io import show, output_file
from bokeh.models import Legend, LegendItem, Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, SaveTool, BoxZoomTool, WheelZoomTool, PanTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import figure
from bokeh.core.properties import field

from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider

In [3]:
minimo=1e-18
maximo=1e-3

In [4]:
a=pd.read_csv('v2_edges_06_after_cleaning_checkcontrol.csv')
b=a[(a['Raw distance']<maximo)]

a=pd.read_csv('edges_06_all.csv')
b=a[(a['Raw distance']==a.iloc[761,2])&(a['Source']!=a['Target'])&(a['Raw distance']>=minimo)&(a['Raw distance']<maximo)]
b.reset_index(inplace=True,drop=True)

In [5]:
len(b)

160

In [6]:
new=b.copy()
new.loc[:,'Source']=new['Source'].apply(lambda x:x.split('.')[0]+'.1')
new.loc[:,'Target']=new['Target'].apply(lambda x:x.split('.')[0]+'.1')

In [7]:
bex=new.copy()
bex=bex[['Source','Target']]
bex.reset_index(inplace=True,drop=True)

In [8]:
#bex[bex.isnull().any(axis=1)]
#comp=bex.apply(np.sort, axis = 1).drop_duplicates()

In [9]:
comp=bex.apply(lambda r: sorted(r), axis = 1).drop_duplicates()
comp2=comp[comp['Target']!=comp['Source']]
comp3=comp2.reset_index(drop=True)
#inflexion

with open('rhodo_dictionary_file_code.pkl', 'wb') as f:
    pickle.dump(din, f)

In [10]:
#with open("rhodo_dictionary_file_code.pkl","rb") as p:
#    din=pickle.load(p)

cou=1
for index,row in comp3.iterrows():
    print(row['Source'],row['Target'])
    pix=0
    for key,value in din.items():    
        if row['Source'] in value:
            sou=key
            pix=pix+1
        if row['Target'] in value:
            tar=key
            pix=pix+1
        if pix==2:
            break
    print(sou,tar)
    del sou,tar
    print(cou)
    cou=cou+1

In [11]:
#results=Parallel(n_jobs=4)(delayed(analisis)(row,index) for index,row in comp3.iterrows())

with open('valuesANIb.pkl', 'wb') as f:
    pickle.dump(results, f)

In [12]:
with open("valuesANIb_cleaning.pkl","rb") as p:
    values=pickle.load(p)

In [13]:
sono=[i[0] for i in values]
tano=[i[1] for i in values]
val=[i[2] for i in values]

In [14]:
valores=pd.DataFrame({'Source':sono,'Target':tano,'ANIb':val})

In [15]:
#valorestmp=valores.copy()

valorestmp['sourcegenome']=valorestmp['Source'].apply(lambda x:dr1[dr2[x]])
valorestmp['targetgenome']=valorestmp['Target'].apply(lambda x:dr1[dr2[x]])

valorestmp2=valorestmp[['sourcegenome','targetgenome','ANIb']].copy()

valorestmp3=valorestmp2.drop_duplicates().copy()

In [16]:
#valorestmp3.to_excel('diccionario_ani_cleaning.xlsx',index=False)

In [17]:
comp4=comp3.merge(valores,how='left')
#comp3=comp3.assign(ANIb=[i[1] for i in values])
comp4.columns=['genome1','genome2','ANIb']

In [18]:
h=b.copy()

In [19]:
h = h.assign(genome1=h['Source'].apply(lambda x:x.split('.')[0]+'.1'))
h = h.assign(genome2=h['Target'].apply(lambda x:x.split('.')[0]+'.1'))

In [20]:
b2=h.merge(comp4,how='outer',on=['genome1','genome2'])
b3=b2.copy()
b3.drop(['genome1','genome2'],axis=1,inplace=True)

In [21]:
b3.loc[b3.isnull().any(axis=1),['ANIb']]=1

In [22]:
b3.columns=['Source','Target','Distance','ANIb']

In [23]:
tempo98=b3.loc[b3['ANIb']>0.98]
#forbid98=tempo98['Target'].tolist()

In [24]:
tempo98.sort_values('ANIb')

,Source,Target,Distance,ANIb
6,NZ_NOZK01000025.cluster017,NZ_NOZK01000025.cluster020,4.163336e-17,1.0
7,NZ_NOZK01000025.cluster018,NZ_NOZK01000025.cluster019,4.163336e-17,1.0
23,NZ_LYPG01000001.cluster004,NZ_LYPG01000001.cluster005,4.163336e-17,1.0
24,NZ_NOYS01000019.cluster011,NZ_NOYS01000019.cluster012,4.163336e-17,1.0
25,NZ_NOYS01000019.cluster003,NZ_NOYS01000019.cluster004,2.955083e-04,1.0


analisis=tempo98.copy()

analisis=analisis[analisis['Source']!=analisis['Target']].sort_values('ANIb')

analisis2=analisis[['Source','Target','ANIb']].copy()

analisis2.loc[:,'Source']=analisis2['Source'].apply(lambda x:x.split('.')[0]+'.1')
analisis2.loc[:,'Target']=analisis2['Target'].apply(lambda x:x.split('.')[0]+'.1')

analisis3=analisis2[analisis2['Source']!=analisis2['Target']].drop_duplicates()

with open('rhodo_assembly_name.pkl','rb') as d1:
    dr1=pickle.load(d1)
with open('rhodo_contig_genome_dict.pkl','rb') as d2:
    dr2=pickle.load(d2)

gt=nx.from_pandas_edgelist(analisis3,source='Source',target='Target')

def contarcontig(namefile):
    cou1=[1 for yi in SeqIO.parse(namefile,'genbank')]
    cou2=[len(yi) for yi in SeqIO.parse(namefile,'genbank')]
    contigs=sum(cou1)
    largo=sum(cou2)
    return contigs,largo

fg=[]
lg=[]
for fr in nx.nodes(gt):
    do,lag=contarcontig('rhodococcusall/'+dr2[fr]+'_genomic.gbff')
    fg.append(do)
    lg.append(lag)
    

f=lambda x:dr1[dr2[x]]

nx.set_node_attributes(gt, dict(zip(nx.nodes(gt),fg)),'contigs')
nx.set_node_attributes(gt, dict(zip(nx.nodes(gt),list(map(f,nx.nodes(gt))))),'nombre')
nx.set_node_attributes(gt, dict(zip(nx.nodes(gt),lg)),'largo')

#nx.get_node_attributes(gt,'nombre')

ghu=pd.read_csv('Lista_final_editada_20190128.csv',sep=';',header=None)

ghusolo=ghu.iloc[:,0]

ghusolo2=ghusolo.apply(lambda x:dr1[x[0:-8]]).tolist()

gt2=gt.subgraph([x for x,y in gt.nodes(data=True) if not(y['nombre'] in ghusolo2)]).copy()

#analisis2['sourcegenome']=analisis2['Source'].apply(lambda x:dr1[dr2[x]])
#analisis2['targetgenome']=analisis2['Target'].apply(lambda x:dr1[dr2[x]])

#analisis3=analisis2[['sourcegenome','targetgenome','ANIb']]

#analisis4=analisis3.drop_duplicates().copy()

#analisis4.to_excel('listasobre98_154genomas.xlsx',index=False)

for fr in nx.nodes(gt):

ansis=pd.unique(analisis['Source'].append(analisis['Target']))

ansis

In [25]:
def retrievebest(dictio):
    nog=[]
    maximo=max(list(dictio.values()))
    for key, value in dictio.items():
        if value==maximo:
            nog.append(key)
    return nog

In [26]:
def iscomplete(graph):
    nn=nx.number_of_nodes(graph)
    ne=nx.number_of_edges(graph)
    if nn*(nn-1)/2==ne:
        return True
    else:
        return False 

In [27]:
q=nx.from_pandas_edgelist(tempo98,source='Source',target='Target',edge_attr='Distance')
tmpg=sorted(nx.connected_component_subgraphs(q),key=len,reverse=True)
listcon=sorted(nx.connected_components(q), key = len, reverse=True)

In [28]:
complete=[]
noncomplete=[]
for i in tmpg:
    nn=nx.number_of_nodes(i)
    ne=nx.number_of_edges(i)
    if nn*(nn-1)/2==ne:
        complete.append(i.copy())
    else:
        noncomplete.append(i.copy())   

In [29]:
complete=sorted(complete,key=len,reverse=True)

In [30]:
noncomplete=sorted(noncomplete,key=len,reverse=True)

In [31]:
print(len(complete))
print(len(noncomplete))
for nt in complete:
    print(len(nx.nodes(nt)))
for nt in noncomplete:
    print(len(nx.nodes(nt)))

5
0
2
2
2
2
2


In [32]:
saved=[]
cou1=1
for ty in tmpg:
    dicyo={}
    for i in list(ty.nodes()):
        qx=SeqIO.read('clusters/'+i+'.gbk','genbank')
        largo=len(qx)
        dicyo[i]=largo
    sob=random.choice(retrievebest(dicyo))
    comprob=(len(set(list(dicyo.values())))==1)
    print(comprob)
    if not(comprob):
        print(dicyo)
        print(cou1)
        cou1=cou1+1
    saved.append(sob)

True
True
True
False
{'NZ_NOYS01000019.cluster011': 40691, 'NZ_NOYS01000019.cluster012': 40690}
1
True


In [33]:
len(set(saved))

5

In [34]:
elimin=set(nx.nodes(q)).difference(set(saved))

In [35]:
elimin

{'NZ_LYPG01000001.cluster005',
 'NZ_NOYS01000019.cluster004',
 'NZ_NOYS01000019.cluster012',
 'NZ_NOZK01000025.cluster017',
 'NZ_NOZK01000025.cluster018'}

saved=[]
cou1=1
for ty in complete:
    dicyo={}
    for i in list(ty.nodes()):
        qx=SeqIO.read('clusters/'+i+'.gbk','genbank')
        largo=len(qx)
        dicyo[i]=largo
    sob=max(dicyo.items(), key=operator.itemgetter(1))[0]
    print(len(set(list(dicyo.values())))==1)
    if len(set(list(dicyo.values())))!=1:
        print(dicyo)
        print(cou1)
        cou1=cou1+1
    saved.append(sob)

saved2=[]
cou2=1
for grafos in noncomplete:
    dicro={}
    for i in list(grafos.nodes()):
        qy=SeqIO.read('clusters/'+i+'.gbk','genbank')
        largo=len(qy)
        dicro[i]=largo
    tob=max(dicyo.items(), key=operator.itemgetter(1))[0]
    print(len(set(list(dicro.values())))==1)
    if len(set(list(dicro.values())))!=1:
        print(dicro)
        print(cou)
        cou2=cou2+1
    saved2.append(tob)

cou=0
saved2=[]
for grafos in noncomplete:
    if cou!=0:
        tmp=grafos.copy()
        s=sorted(tmp.degree(), key=lambda x: x[1], reverse=True)
        listado=[i[0] for i in s]
        for j in listado:
            vecinos=list(tmp.neighbors(j))
            nodost=list(tmp.nodes())
            nodost.remove(j)
            if set(vecinos)==set(nodost):
                #print(j)
                print('si')
                saved2.append(j)
                break
    cou=cou+1

dic={}
cou=0
for grafos in noncomplete:
    tmp=grafos.copy()
    s=sorted(tmp.degree(), key=lambda x: x[1], reverse=False)
    print(s)
    saved=[]
    bl=[]
    for n in s:
        bl=list(set(bl+list(tmp.neighbors(n[0]))))
        #print(bl)
        if not(n[0] in bl):
            saved.append(n[0])
        tmp.remove_node(n[0])
        if iscomplete(tmp):
            q=list(set(nx.nodes(tmp))-set(bl))
            if q!=[]:
                selec=random.choice(q)
                saved.append(selec)
            print(1)
            break
        else:
            print(0)
    dic[cou]=saved
    cou=cou+1

In [36]:
#G=reduce(lambda x,y:nx.compose(x.copy(),y.copy()),noncomplete)

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"

plot.add_tools(HoverTool(tooltips=[('id','@index'),('contigs','@contigs'),('largo','@largo'),('bacteria','@nombre')]), TapTool(), BoxSelectTool(),SaveTool(),BoxZoomTool(),WheelZoomTool(), PanTool())

graph_renderer = from_networkx(gt2, nx.spring_layout, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(size=10, fill_color='red')
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
#graph_renderer.selection_policy = EdgesAndLinkedNodes()
graph_renderer.inspection_policy = NodesAndLinkedEdges() #
plot.renderers.append(graph_renderer)

#print(plot.renderers)
#li1=LegendItem(label=field('phage'),renderers=[plot.renderers[1].node_renderer])
#li2=Legend(items=[li1])
#plot.add_layout(li2)

output_file("interactive_graphs.html")

show(plot)

In [37]:
newedges=a.loc[(~a['Source'].isin(elimin))&(~a['Target'].isin(elimin))]

In [38]:
len(a)

37948

In [39]:
len(newedges)

37757

In [40]:
newedges.to_csv('v2_edges_06_after_cleaning_checkcontrol_dereplication.csv',index=False)

In [41]:
filenode=pd.read_csv('v2_nodes_after_cleaning_checkcontrol.csv')

In [42]:
newnodes=filenode.loc[~filenode['Id'].isin(elimin)]

In [43]:
newnodes.to_csv('v2_nodes_after_cleaning_checkcontrol_dereplication.csv',index=False)

In [44]:
tuyo=[]
for tu in newnodes['Id']:
    edx=SeqIO.read('clusters/'+tu+'.gbk','genbank')
    larg=len(edx)
    tuyo.append(larg)

In [45]:
newnodes2=newnodes.copy()
newnodes2['length']=tuyo

In [46]:
newnodes2.to_excel('v2_nodes_after_cleaning_checkcontrol_dereplication.xlsx',index=False)

In [47]:
pikachu=newnodes['reduccion'].value_counts()

In [ ]:
pikachu.to_excel('conteo_154genomas+BGC_check_control+dereplication.xlsx')

In [ ]:
pikachu

In [ ]:
source=[]
target=[]
for par1,par2 in itertools.combinations(nx.nodes(G),2):
    source.append(par1)
    target.append(par2)

In [ ]:
nuevo=pd.DataFrame({'Source':source,'Target':target})
nuevo=nuevo.apply(lambda r: sorted(r), axis = 1)

In [ ]:
nuevo2=nuevo.merge(a,how='inner')

In [ ]:
nuevo2.columns=['Source','Target','Distance']

In [ ]:
zz=nx.from_pandas_edgelist(nuevo2,source='Source',target='Target',edge_attr='Distance')

In [ ]:
tmpg=sorted(nx.connected_component_subgraphs(zz),key=len,reverse=True)

In [ ]:
noto=[]
for yu in tmpg:
    #print(nx.nodes(yu))
    print(nx.get_edge_attributes(yu,'Distance'))
    tu=set(nx.get_edge_attributes(yu,'Distance').values())
    if len(tu)!=1:
        noto.append(yu)

In [ ]:
G2=reduce(lambda x,y:nx.compose(x,y),noto) 

In [ ]:
for neo in [i for i in nx.nodes(tmpg[7])]:
    copyfile('anti_folders/clusters/'+neo+'.gbk','transitivity/'+neo+'.gbk')

q=nx.from_pandas_edgelist(tempo95,source='Source',target='Target')
listcon=sorted(nx.connected_components(q), key = len, reverse=True)
rancho95=[]
for i in listcon:
    rancho95.append(random.choice(list(i)))
forbid95=list(set(list(q.nodes()))-set(rancho))

In [ ]:
with open('forbid98.pkl', 'wb') as f:
    pickle.dump(forbid98, f)

with open('forbid95.pkl', 'wb') as f:
    pickle.dump(forbid95, f)

with open("forbid95.pkl","rb") as p:
    forbid95=pickle.load(p)

In [ ]:
with open("forbid98.pkl","rb") as q:
    forbid98=pickle.load(q)

In [ ]:
final95=a.loc[(a['Raw distance']!=0) & (~a['Source'].apply(lambda x:x in forbid95))&(~a['Target'].apply(lambda x:x in forbid95))]

In [ ]:
final98=a.loc[(a['Raw distance']!=0) & (~a['Source'].apply(lambda x:x in forbid98))&(~a['Target'].apply(lambda x:x in forbid98))]

In [ ]:
len(pd.unique(final98['Source'].append(final98['Target'])))

In [ ]:
len(rancho98)

In [ ]:
final95.to_csv('final95_0_6.csv',index=False)
final98.to_csv('final98_0_6.csv',index=False)

In [ ]:
nodos=pd.read_csv('nodes_all_modified.csv')

In [ ]:
hcaf8=nodos[nodos['Description'].str.contains("H-CA")]['Id'].tolist()

In [ ]:
newtable=nodos.loc[~nodos['Id'].apply(lambda x: x in forbid98)]

In [ ]:
newtable.to_csv('nodes_all_modified_derreplicated.csv',index=False)

In [ ]:
ty=newtable['reduccion'].value_counts()

In [ ]:
len(pd.unique(newtable['Id'].apply(lambda x: x[0:-11])))

In [ ]:
ty2=nodos['reduccion'].value_counts()

In [ ]:
ty2.sum()